# Notebook zum Finden der richtigen Mosmix-Stationen für eine Region

In [17]:
#Imports
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point, Polygon
import csv
import os
import configparser

In [25]:
# Parameter aus config

# get the current script directory path
currentScriptDirectoryPath = "pathto/mosmix_map_generator"  # geht im notebook nur absolut anscheinend
configFilePath = os.path.join(currentScriptDirectoryPath, 'config.ini')

# init configParser and open file
config = configparser.ConfigParser()
config_result = config.read(configFilePath, encoding='utf8')

# SET Parameters
alle_mosmix_station_file = os.path.join(currentScriptDirectoryPath, config.get('pfade_stationen', 'alle_mosmix_station_file'))
mosmix_ids_vom_server_file = os.path.join(currentScriptDirectoryPath, config.get('pfade_stationen', 'mosmix_ids_vom_server_file'))
nuts1 = os.path.join(currentScriptDirectoryPath, config.get('pfade_stationen', 'nuts1'))
stationen_file = os.path.join(currentScriptDirectoryPath, config.get('pfade', 'stationen_file'))

### IDs abgleichen (Stationslexikon vs Schnittstelle)
#### Außerdem Stationen als Punkte einladen, damit im nächsten Schritt geographisch gefiltert werden kann

In [20]:
# Aus Stationslexikon
txtfile = open(alle_mosmix_station_file, "r", encoding = 'UTF-8')
stationen = txtfile.read()
stationen = stationen.split('\n')
stationsliste = []
stations_ids_lexikon = []
for line in stationen:
    number = line[:5].strip()
    name = line[10:32].strip()
    # Koordinaten sind in Grad und Minuten gegeben (?!#?!), umrechnen in Dezimalgrad
    latitude = float(line[32:39].strip())
    grad = int(latitude)
    minuten = (latitude - grad) * 100
    latitude = round(grad + minuten / 60.0,2)
    longitude = float(line[40:47].strip())
    grad = int(longitude)
    minuten = (longitude - grad) * 100
    longitude = round(grad + minuten / 60.0, 2)

    stationsliste.append([number, name, latitude, longitude])
    stations_ids_lexikon.append(number)

# Aus Schnittstelle zum Abrufen der kml-Dateien
txtfile = open(mosmix_ids_vom_server_file, "r", encoding = 'UTF-8')
stationen = txtfile.read()
stationen = stationen.split('\n')
stations_ids_schnittstelle = []
for line in stationen:
    id = line.split('/')[0]
    stations_ids_schnittstelle.append(id)

# Auswertung ergibt, dass alle Stationen, die in Frage kommen, in beiden Verzeichnissen vorhanden sind - Somit können alle Stationen auch genutzt werden
# print(len(stations_ids_lexikon))
# print(len(stations_ids_schnittstelle))
# for i in stations_ids_schnittstelle:
#     if i in stations_ids_lexikon:
#         print('Ja ', i)
#     else:
#         print('Nein ', i)

# for i in stationsliste:
#     print(i)

### Geographisch filtern
#### Hier nur Stationen in Schleswig-Holstein oder Umgebung nutzen

In [21]:
#### Stationen filtern ####

# Nur Stationen verwenden, die in einem iterativ bestimmten Kompartment um Schleswig-Holstein liegen
# Um die Landesgrenze herum wird das Shape so bestimmt, dass eine konvexe Hülle aus den Stationen Schleswig-Holstein komplett einhüllen würde

shape_zum_filtern = Polygon()

gefilterte_ids = []
gefilterte_stationen = []

sueden, norden = 53.25, 55.3
westen, osten = 7.5, 11.5
kompartment = Polygon((Point(westen,norden), Point(osten,norden), Point(11.8,54.2), Point(11.3,53.8), Point(11,53.3), Point(10.4,sueden), Point(10,53.5), Point(9.6,53.55), Point(9.1,53.75), Point(westen,54.1)))
for row in stationsliste:
    punkt = Point(row[3],row[2])
    if kompartment.contains(punkt):
        # print(row[2], (8 - len(str(row[2])))*" ", row[3])
        gefilterte_stationen.append(row)
        gefilterte_ids.append(row[0])

# In GDF umwandeln
df = pd.DataFrame(gefilterte_stationen, columns=['id', 'name', 'latitude', 'longitude'])
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
stationen_gdf = gpd.GeoDataFrame(df, geometry=geometry)

stationen_gdf.set_crs(epsg=4326, inplace=True)

# print(stationen_gdf)

# for row in stationen_gdf.iterrows():
#     print(row)

,id,name,latitude,longitude,geometry
0,06110,SKRYDSTRUP,55.23,9.27,POINT (9.27 55.23)
1,06119,KEGNAES,54.85,9.98,POINT (9.98 54.85)
2,06141,ABED,54.83,11.33,POINT (11.33 54.83)
3,10015,HELGOLAND,54.17,7.88,POINT (7.88 54.17)
4,10018,WESTERLAND,54.92,8.35,POINT (8.35 54.92)
...,...,...,...,...,...
125,X134,SWIS-PUNKT,54.38,10.15,POINT (10.15 54.38)
126,X137,SWIS-PUNKT,53.80,10.72,POINT (10.72 53.8)
127,X149,TINNUM,54.90,8.33,POINT (8.33 54.9)
128,X391,BOLTENHAGEN,54.00,11.18,POINT (11.18 54)


### Alles in einer Karte darstellen
#### Das Polygon zum Ausschneiden wurde iterativ bestimmt.
#### Die convexe Hülle veranschaulicht in welchem Bereich die Wetterkarten erstellt werden.

In [23]:
# Landesgrenze Schleswig-Holstein einladen
nuts1 = gpd.read_file(nuts1, layer = 'nuts5000_n1')
sh = nuts1[nuts1['NUTS_NAME'] == 'Schleswig-Holstein']
sh = sh.to_crs(epsg=4326)

# Erstellen Basiskarte
m = folium.Map(location=[sh.geometry.centroid.y.mean(), sh.geometry.centroid.x.mean()], zoom_start=8)

# Hinzufügen der Landesgrenze zur Karte
for _, row in sh.iterrows():
    sim_geo = gpd.GeoSeries(row['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j)
    geo_j.add_to(m)

# Hinzufügen der Stationen zur Karte
for idx, row in stationen_gdf.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"ID: {row['id']}<br>Name: {row['name']}<br>latitude: {row['latitude']}<br>longitude: {row['longitude']}",
        icon=folium.Icon(color='yellow', icon='info-sign')
    ).add_to(m)

# Hinzufügen des Kompartments zur Karte
kompartment_json = gpd.GeoSeries(kompartment).simplify(tolerance=0.001).to_json()
folium.GeoJson(data=kompartment_json, style_function=lambda feature: {
        "fill": False,
        "color": "black",
        "weight": 2,
        "dashArray": "5, 5",
    },
).add_to(m)

# Konvexe Hülle 
konvexe_hull = stationen_gdf.unary_union.convex_hull
folium.GeoJson(data=gpd.GeoSeries(konvexe_hull).simplify(tolerance=0.001).to_json(), style_function=lambda feature: {
        "fill": False,
        "color": "green",
        "weight": 2,
        "dashArray": "5, 5",
    },
).add_to(m)


# Legende hinzufügen
legend_html = """
<div style="
    position: fixed; 
    bottom: 50px; left: 50px; width: 200px; height: 140px; 
    background-color: white; z-index:9999; font-size:14px;
    border:2px solid grey; border-radius:6px; padding: 10px;">
    <b>Legende</b> <br>
    <i class="fa fa-info-circle" style="color:red"></i> Stationen <br>
    <svg width="20" height="10">
        <line x1="0" y1="5" x2="20" y2="5" style="stroke:green;stroke-width:2;stroke-dasharray:5,5" />
    </svg> Konvexe Hülle <br>
    <svg width="20" height="10">
        <line x1="0" y1="5" x2="20" y2="5" style="stroke:black;stroke-width:2;stroke-dasharray:5,5" />
    </svg> Kompartment <br>
</div>
"""

m.get_root().html.add_child(folium.Element(legend_html))

m

/var/folders/zm/bggcfx013jx13h3z_69vcv0r0000gn/T/ipykernel_26036/1799201171.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[sh.geometry.centroid.y.mean(), sh.geometry.centroid.x.mean()], zoom_start=8)
/var/folders/zm/bggcfx013jx13h3z_69vcv0r0000gn/T/ipykernel_26036/1799201171.py:21: UserWarning: color argument of Icon should be one of: {'orange', 'darkblue', 'lightblue', 'red', 'darkgreen', 'lightgray', 'white', 'green', 'darkpurple', 'lightred', 'cadetblue', 'gray', 'black', 'pink', 'darkred', 'blue', 'purple', 'beige', 'lightgreen'}.
  icon=folium.Icon(color='yellow', icon='info-sign')
/var/folders/zm/bggcfx013jx13h3z_69vcv0r0000gn/T/ipykernel_26036/1799201171.py:35: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  konvexe_hull = stationen_gdf.unary_union.convex_h

### Schlussendlich die Stationen mit id und Koordinaten in csv schreiben 
#### csv-Datei wird im Datenimport-Skript genutzt um die richtigen Stationen vom OpenData-Portal zu laden, außerdem kommen die Koordinaten aus dem File

In [28]:
#### in csv schreiben für Importskript ####

header = ['Stationsname','id','lat','lon']
with open(stationen_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(header)

    for index, row in stationen_gdf.iterrows():
        writer.writerow([row['name'], row['id'], row['latitude'], row['longitude']])